In [ ]:
from src.data_structures import *
from ortools.sat.python import cp_model

# Create the data

Given a total of $L$ layers, we know the following information for each layer $l \in [0 \ldots L]$:

- $m(l)$ = memory consumption of layer $l$;
- $t(l)$ = execution time of layer $l$.
- $o(l)$ : output memory size of layer $l$.
 
Given a total of $S$ execution segments, we know the following information for each segment $s \in [0 \ldots S]$:

- $M(s)$ = the available memory of segment $s$;
- $R(s)$ = the available time of segment $s$.
 

In [ ]:
# Initialize the layers.
layers = [
    Layer(id=0, memory=3, runtime=1, output_size=2),
    Layer(id=1, memory=4, runtime=1, output_size=3),
    Layer(id=2, memory=5, runtime=1, output_size=4),
    Layer(id=3, memory=3, runtime=1, output_size=2),
    Layer(id=4, memory=4, runtime=1, output_size=3),
    Layer(id=5, memory=2, runtime=1, output_size=1),
]

# Initialize the execution segments.
segments = [
    Segment(id=0, avail_memory=12, avail_time=6),
    Segment(id=1, avail_memory=8, avail_time=6),
    Segment(id=2, avail_memory=6, avail_time=6),
    Segment(id=3, avail_memory=12, avail_time=6),
    Segment(id=4, avail_memory=4, avail_time=6),
    Segment(id=5, avail_memory=5, avail_time=6),
    Segment(id=6, avail_memory=6, avail_time=6),
    Segment(id=7, avail_memory=8, avail_time=6)
]

# Pre-compute some sizes, and indices.
num_layers = len(layers)
all_layers = range(num_layers)
num_segments = len(segments)
all_segments = range(num_segments)
max_output_size = max([layer.output_size for layer in layers])


# Declare the MIP solver

In [ ]:
model = cp_model.CpModel()

# Create the variables
What follows is the list of decision variables for our problem.

The first variable we define is $x$, which keeps track of where the layers are placed as follows:
$$
x[l, s] =
\begin{cases}
1, & \text{if layer $l$ is placed in segment $s$} \\
0, & \text{otherwise}
\end{cases}\\
\forall l \in [0 \ldots L], \forall s \in [0 \ldots S]
$$

The second variables is $u$, which keeps track of wheter a segment is in use as follows:
$$
u[s] =
\begin{cases}
1, & \text{if segment $s$ contains at least a layer} \\
0, & \text{otherwise}
\end{cases}\\
\forall s \in [0 \ldots S]
$$

Then we define $i$, which contains the index of the last layer placed inside a segment as follows:
$$
0 \leq i[s] \leq L, \qquad \forall s \in [0 \ldots S]
$$

Finally, $y$ keeps track of the output size of the last layer placed inside a segment as follows:
$$
y[s] \geq 0, \qquad \forall s \in [0 \ldots S]
$$


In [ ]:
# Initialize the decision variable x[l, s].
x = {}
for l in all_layers:
    for s in all_segments:
        x[l, s] = model.NewBoolVar(f"x_{l}_{s}")

# Determines if a segment is in use.
u = []
for s in all_segments:
    u.append(model.NewBoolVar(name=f"u_{s}"))

# This decision variable will act as an index to the last layer placed in an
# execution segment.
i = []
for s in all_segments:
    i.append(model.NewIntVar(lb=0, ub=num_layers, name=f"i_{s}"))

# This decision variable will be set to the output size of the last layer
# placed in an execution segment.
y = []
_y = []
for s in all_segments:
    y.append(model.NewIntVar(lb=0, ub=max_output_size, name=f"y_{s}"))
    _y.append(model.NewIntVar(lb=0, ub=max_output_size, name=f"_y_{s}"))

# We assemble in an array the output sizes for all layers. We need the first
# zero because the default value for the index i is 0, thus, by default it would
# select the output size of the first layer. Even when no layer is placed inside
# a segment.
output_sizes = [layers[l].output_size for l in all_layers]


# Define the constraints
The first constraint ensures that we place a layer in only one segment:
$$
    \sum_{s=0}^{S} x[l, s] = 1
    \quad
    \forall l \in [0 \ldots L]
$$

The next constraint ensures that the total memory occupied by the layers placed inside a segment is lower than the segment's capacity:
$$
    \sum_{l=0}^{L} x[l, s] \cdot m(l) \le M(s),
    \quad
    \forall s \in [0 \ldots S]
$$

The same applies for the runtime of a layer:
$$
    \sum_{l=0}^{L} x[l, s] \cdot t(l) \le T(s),
    \quad
    \forall s \in [0 \ldots S]
$$

With the following constraint, $u$ becomes $true$ only if there are layers placed inside the given segment:
$$
    u[s] = \max_{l \in [0 \ldots L]} \quad x[l, s],
    \quad
    \forall s \in [0 \ldots S]
$$
Being $x$ a boolean variable, taking the maximum of a set of $x$ variables results in a boolean.

The next constraints ensures that if we do not use a segment $s$, we are not going to use the next segments:
$$
    \neg u[s] \implies \neg u[s + 1],
    \quad
    \forall s \in [0 \ldots S - 1]
$$
This ensures that the segments are incrementally used, from the first to the last, without jumps (e.g., using segments 0, 6, 12, and so forth).

The next constraint ensures that we execute the layers of a neural network in an ordered fashion:
$$
    x[l_{1}, s_{1}] \implies \neg x[l_{2}, s_{2}],\\
    \forall s_{1} \in [0 \ldots S],
    \forall s_{2} \in [s_{1} + 1 \ldots S],\\
    \forall l_{1} \in [0 \ldots L],
    \forall l_{2} \in [0 \ldots l_{1}]
$$
To better understand this constraint, let us make an example with 4 layers and 4 segments:
If layer 2 is placed in segment 2, the previous layers (i.e., 0 and 1) cannot be placed in the next execution segments (i.e., 3 and 4).

We are going to keep track of the index of the last layer placed inside a segment, as follows:
$$
    i[s] = \max_{l \in [0 \ldots L]} \quad x[l, s] \cdot l,
    \quad
    \forall s \in [0 \ldots S],
$$

Finally, we use the previously computed index, to get the actual output size of that layer, if and only if the execution segment is actually in use, as follows:
$$
    y[s] = o(i[s]) \cdot u[s]
    \quad
    \forall s \in [0 \ldots S],
$$

In [ ]:
# Each layer is assigned to exactly one execution segment.
for l in all_layers:
    model.AddExactlyOne(x[l, s] for s in all_segments)

# The amount of occupied memory in each execution segment cannot exceed its memory.
for s in all_segments:
    model.Add(sum(x[l, s] * layers[l].memory for l in all_layers) <= segments[s].avail_memory)

# The runtime for the layers executed in each execution segment cannot exceed its duration.
for s in all_segments:
    model.Add(sum(x[l, s] * layers[l].runtime for l in all_layers) <= segments[s].avail_time)

# If there is even one layer assigned to a segment, u will be the max of a series
# of zeros and a one.
for s in all_segments:
    model.AddMaxEquality(u[s], [x[l, s] for l in all_layers])

# If later `s` is not used, then then next later cannot be in use.
for s in range(0, num_segments - 1):
    model.AddImplication(u[s].Not(), u[s + 1].Not())

# The layers are placed in an ordered fashion inside the sequences.
for s0 in all_segments:
    for l0 in all_layers:
        for s1 in range(s0 + 1, num_segments):
            for l1 in range(0, l0):
                model.AddImplication(x[l0, s0], x[l1, s1].Not())

# The index variable, will be assigned the index of the last layer placed inside
# a segment. We use (l + 1) because the first element at index 0, is a placeholder
# of value 0.
for s in all_segments:
    model.AddMaxEquality(i[s], [x[l, s] * l for l in all_layers])

# With this one, we basically implement:
#   y[s] = output_sizes[i[s]] * u[s]
# When writing a CP problem, we canno simply use a decision variable as index,
# we need to do a couple of extra steps, as you might have noticed. Furthermore,
# we conside the output size only if the segment is in use.
for s in all_segments:
    model.AddElement(i[s], output_sizes, _y[s])
    model.AddMultiplicationEquality(y[s], [_y[s], u[s]])


# Define the objective and invoke the solver
Now, our optimization function aims at minimizing the sum of the output size of the last layers placed inside the execution segments:
$$
    \text{minimize}\quad\sum_{s \in [0 \ldots S]} y[s]
$$

In [ ]:
# The following code defines the objective function for the problem. In our case
# we want to minimize the **memory to store** in memory by the **last layer**
# placed **inside an execution segment**.
model.Minimize(sum(y[s] for s in all_segments))

# Create the solver and run it on the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status != cp_model.OPTIMAL:
    print("The problem does not have an optimal solution.")
    exit(1)

# Initialize the solution.
allocations = []
for s in all_segments:
    layers_in_segment = [layers[l] for l in all_layers if solver.Value(x[l, s])]
    if layers_in_segment:
        allocations.append(Allocation(segments[s], layers_in_segment))

# Print solution

In [ ]:
print_assignment_statistics(allocations)

# Compare to Knapsack

In [ ]:
import matplotlib.pyplot as plt

import src.optimizer as optimizer
import src.plot_support as plotter

In [ ]:
knapsack = optimizer.knapsack(layers, segments, True)
print_assignment_statistics(knapsack)
